In [164]:
# Schoof's algorithm v.2

# TO-DO:
#  - replace (x^3 + a*x + b) with f everywhere
#  - optimize some polynomial calculations
#  - use positive integers in CRT
#  - doesnt work for low q (B wont reach the upper bound)
#  - create cache for f_l_bar
#  - possibility for q=p^k
#  - check if gcd is always 1 (vs an invertible element of F_q[x])
#  - does it really not work over Z_5?


def get_s_bar(l):  # pg.47
    q_l = q % l
    
    if q_l == 1:
        return x^(q^2) - x
    
    f_plus0 = get_f_bar(q_l)
    f_minus1 = get_f_bar(q_l - 1)
    f_plus1 = get_f_bar(q_l + 1)
    
    if q_l % 2 == 0:
        return 4*(x^(q^2) - x)*f*f_plus0^2 + f_minus1*f_plus1
    else:
        return (x^(q^2) - x)*f_plus0^2 + 4*f*f_minus1*f_plus1

def get_f_bar(m):  # pg.38
    if m < 0:
        print("Error, m is negative:", m)
    
    cache_info[m] += 1
    
    if m == 0:
        return 0
    elif m == 1:
        return 1
    elif m == 2:
        return 1
    elif m == 3:
        return 3*x^4 + 6*a*x^2 + 12*b*x - a^2
    elif m == 4:
        return 2*(x^6 + 5*a*x^4 + 20*b*x^3 - 5*a^2*x^2 - 4*a*b*x - 8*b^2 - a^3)
    else:
        m_bar = m // 2
        f_plus0 = get_f_bar(m_bar)
        f_plus1 = get_f_bar(m_bar + 1)
        f_plus2 = get_f_bar(m_bar + 2)
        f_minus1 = get_f_bar(m_bar - 1)
        
        if m % 2 == 1:
            if m_bar % 2 == 1:
                #return f_plus2*f_plus2^3 - 16*f^2*f_minus1*f_plus1^3
                return f_plus2*f_plus0^3 - 16*f^2*f_minus1*f_plus1^3
            else:
                return 16*f^2*f_plus2*f_plus0^3 - f_minus1*f_plus1^3
        else:
            f_minus2 = get_f_bar(m_bar - 2)
            return f_plus0*(f_plus2*f_minus1^2 - f_minus2*f_plus1^2)
    
def get_s(m):  # via pg.47 S.2
    if m == 1:
        return 1
    
    f_plus0 = get_f_bar(m)
    
    if m % 2 == 0:
        return 16*f^2*f_plus0^3
    else:
        return f_plus0^3

def get_r(m):  # via pg.47 S.2
    if m == 1:
        return 1
    
    f_plus1 = get_f_bar(m + 1)
    f_plus2 = get_f_bar(m + 2)
    f_minus1 = get_f_bar(m - 1)
    f_minus2 = get_f_bar(m - 2)
    
    return f_plus2*f_minus1^2 - f_minus2*f_plus1^2

def tyzero(l, q_l):
    s_ql = get_s(q_l)
    r_ql = get_r(q_l)
    mixed = s_ql*f^((q^2 - 1)/2) + r_ql  # via lecture 14 - observation   # incorrect?!?!
    f_bar = get_f_bar(l)
    
    if gcd(f_bar, mixed) == 1:  # exact division?
        return False
    else:
        return True

def eigen(l, gamma, g_l):  # via lecture 14 - fact
    # 1st coordinates equal (we know from the "if" branch of schoof_core) ?
    
    s_gamma = get_s(gamma)
    r_gamma = get_r(gamma)
    mixed = f^((q - 1)/2)*s_gamma - r_gamma
    
    if gcd(g_l, mixed) == 1:
        return False
    else:
        return True

def equalx(l, g_l):
    F_l = GF(l)
    
    q_l = q % l
    if tyzero(l, q_l):
        return 0
    
    tau = sqrt(F_l(4 * q_l))  # over F_l
    gamma = 2 * q_l / tau  # over F_l
    
    tau = ZZ(tau)
    gamma = ZZ(gamma)
    if eigen(l, gamma, g_l):
        return tau
    return - tau

def get_c(m):  # via pg.46 S.1
    if m == 1:
        return 0
    
    f_plus1 = get_f_bar(m + 1)
    f_minus1 = get_f_bar(m - 1)
    
    if m % 2 == 1:
        result = 4*f*f_minus1*f_plus1
    else:
        result = f_minus1*f_plus1
    
    return result

def get_d(m):  # via pg.46 S.1
    if m == 1:
        return 1
    
    f_plus0 = get_f_bar(m)
    
    if m % 2 == 1:
        result = f_plus0^2
    else:
        result = 4*f*f_plus0^2
        
    return result

def get_h_X(l, tau, f_l_bar):
    q_l = q % l
    
    c_ql = get_c(q_l)
    d_ql = get_d(q_l)
    r_ql = get_r(q_l)
    s_ql = get_s(q_l)
    
    c_tau = get_c(tau)
    d_tau = get_d(tau)
    
    c_tau = R(c_tau)  # interpet as element of F_q[x] so we could evaluate at x^q
    d_tau = R(d_tau)
    #print("c,d:", c_tau, d_tau)
    #print("deg c,d:", c_tau.degree(), d_tau.degree())
    #if d_tau.degree() > 0:
        #print("smth")
        #print(c_tau)
    
    den0 = (s_ql*(d_ql*(x^(q^2) - x) + c_ql))^2 #% f_l_bar
    den1 = d_ql #% f_l_bar
    den2 = d_tau(x^q) #% f_l_bar
    
    den01 = den0 * den1
    den02 = den0 * den2
    den12 = den1 * den2
    
    den012 = den01 * den2
    
    num0 = f * (d_ql*(f^((q^2 - 1)/2) * s_ql - r_ql))^2 * den12 #% f_l_bar
    num1 = c_ql * den02 #% f_l_bar
    num2 = c_tau(x^q) * den01 #% f_l_bar
    rest = (-x^(q^2)-x-x^q)*den012 #% f_l_bar
    
    final = num0 + rest + num1 + num2
    
    return final


def get_h_Y(l, tau, f_l_bar):
    q_l = q % l
    
    c_ql = get_c(q_l)
    d_ql = get_d(q_l)
    r_ql = get_r(q_l)
    s_ql = get_s(q_l)
    
    s_tau = get_s(tau)
    r_tau = get_r(tau)
    
    s_tau = R(s_tau)  # interpet as element of F_q[x] so we could evaluate at x^q
    r_tau = R(r_tau)
    
    den0 = s_ql*(d_ql*(x^(q^2) - x) + c_ql) % f_l_bar
    
    lambda_bar = d_ql * (f^((q^2 - 1)/2) * s_ql - r_ql)
    
    part0 = lambda_bar * (-f*lambda_bar^2*d_ql - c_ql*den0^2 + (2*x^(q^2) + x)*den0^2*d_ql) * s_tau(x^q)
    part1 = f^((q^2 - 1)/2) * den0^3 * d_ql * s_tau(x^q)
    part2 = f^((q - 1)/2) * r_tau(x^q) * den0^3 * d_ql
    
    final = part0 - part1 - part2
    
    return final

def nonequalx(l, tau):
    f_l_bar = get_f_bar(l)
    
    h_X = get_h_X(l, tau, f_l_bar)
    h_Y = get_h_Y(l, tau, f_l_bar)
    
    if gcd(h_X, f_l_bar) == 1:
        return 0
    if gcd(h_Y, f_l_bar) == 1:
        return -1
    return 1

def recover_t(M):
    rems = []
    mods = []
    modulus = 1
    for pair in M:
        rems.append(pair[1])
        mods.append(pair[0])
        modulus *= pair[0]
    result = CRT_list(rems, mods)
    if result > modulus // 2:
        result -= modulus
    return result

def schoof_core():
    P = Primes()
    B = 2
    l = 2
    
    if gcd(x^q-x, f) == 1:
        tau = 1
    else:
        tau = 0
    M = [(2, tau)]
    
    while B < 4 * sqrt(RR(q)):
        l = P.next(l)  # l shouldnt divide q (happens if q=l^k)
        #print("Processing prime:", l)
        B *= l
        s_l_bar = get_s_bar(l)
        f_l_bar = get_f_bar(l)
        #print("f_l_bar:", f_l_bar)
        
        #skip = False
        
        g_l = gcd(s_l_bar, f_l_bar)
        #print("g_l", g_l)
        if g_l != 1:
            tau = equalx(l, g_l)
        else:
            tau = 0
            r = 0
            while r == 0:
                tau += 1
                #print("tau:", tau)
                r = nonequalx(l, tau)
                #if tau > (l-1) / 2:
                #    skip = True
                #    print("break")
                #    tau = 0
                #    break
            #print("after break")
            if r == -1:
                tau = - tau
        
        # test this:
        #print(tau)
        #if tau < 0:
        #    tau = l + tau
        #if skip:
        #    B = B // l
        #else:
        #    M.append((l, tau))
        
        M.append((l, tau))
    t = recover_t(M)
    size = q + 1 - t
    
    #print("t, q, size, B:", t, q, size, B)
    
    # why ?
    #if size < 0:
    #    size = B + size
    
    #print("size:", size)
    #print("hasse lower:", q + 1 - 2*RR(q))
    
    #if not (q + 1 - 2*sqrt(RR(q)) < size < q + 1 + 2*sqrt(RR(q))):
        #print("t, q, B", t, q, B)
    #    t = t - B
    #    size = q + 1 - t
        
    
    return size

def is_smooth(q, a, b):
    if (4*a^3 + 27*b^2) % q == 0:
        return False
    else:
        return True


# global variables:
#q = 23

PP = Primes()

max_prime = 100  # works fine 11->73
q = 7  # char(K) not divisible by 2,3; for q=5 we have 4*sqrt(5)=8.94 > 6=2*3

many_examples = False
#many_examples = True

#print(GF(7)(59))
#print(GF(7)(59).is_square())

if many_examples:
    while q <= max_prime:
        F_q = GF(q)
        R.<x> = PolynomialRing(F_q)
        print("Testing all smooth W. curves over field q:", q)
        for a in range(1, q):
            for b in range(q):
                #q, a, b = 11, 2, 2

                f = x^3 + a*x + b

                #print("Working with curve:", "a=", a, "b=", b, "; over q=", q,)
                smoothness = is_smooth(q, a, b)
                #print("Is the curve smooth?", smoothness)
                #print("q, a, b:", q, a, b, "smooth:", smoothness)

                if smoothness:
                    # check with built-in functions:
                    E = EllipticCurve(GF(q), [0,0,0,a,b])  # a_1,...,a_6 ; a_5~a, a_6~b
                    correct_size = len(list(E))
                    #print("Size of E (from built-in):", len(list(E)))

                    schoof_size = schoof_core()
                    #print("The size of the group (defined with:", "a=", a, "b=", b, ", over q=", q, ") is:", schoof_core())
                    if correct_size == schoof_size:
                        placeholder = 0
                        #print("Correct")
                    else:
                        print("!!!!!!! Incorrect: ", "Schoof :", schoof_size, "Built-in :", correct_size)
        q = PP.next(q)

else:
    q = 1999  # 101, 191, 307, 881, 1999, 4001
    a, b = randint(1, q - 1), randint(0, q - 1)
    #a, b = 3, 22
    F_q = GF(q)
    R.<x> = PolynomialRing(F_q)
    f = x^3 + a*x + b
    smoothness = is_smooth(q, a, b)
    
    cache_info = [0] * q
    if smoothness:
        print("Calculations began for (q, a, b) =", q, a, b)
        schoof_size = schoof_core()
        
        E = EllipticCurve(GF(q), [0,0,0,a,b])  # a_1,...,a_6 ; a_5~a, a_6~b
        correct_size = len(list(E))
        
        print("our vs correct:", schoof_size, correct_size)
        print("Cache:")
        print(cache_info[:20])
        

print("Finished calculations.")



Calculations began for (q, a, b) = 1999 672 176
our vs correct: 1945 1945
Cache:
[1, 18, 23, 29, 27, 14, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Finished calculations.
